# CATEGORICAL ENCODING

### One Hot Encoding

- One-Hot Encoding is the most common method used to convert categorical data into numerical form — specifically, into binary (0/1) variables — so that machine-learning models can process them.

    - For a categorical variable with k unique categories, OHE creates k new columns (also called dummy variables).
    - Each column represents one possible category.
    - In each row, only one of these columns will have a 1 (indicating presence of that category); all others will have 0.
    
- Makes categorical data numeric, so ML models can use it.
- Works especially well with linear models (like logistic regression or linear regression).
- The encoded columns clearly express which category each record belongs to.

- Limitations
    - High cardinality problem:
        - If a variable has many unique categories (e.g., 100 city names), OHE will create 100 new columns — expanding the dataset significantly.
    - Multicollinearity:
        - Some of the dummy variables are highly correlated (for instance, if “Male” = 0, it automatically means “Female” = 1). To avoid this, one dummy variable is usually dropped (called “drop-first” encoding).

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings('ignore')

In [2]:

# Load the dataset

data = pd.read_csv('student_performance_updated_1000.csv')


In [3]:
print("\nDataset Shape:", data.shape)


Dataset Shape: (1000, 12)


In [4]:
data.head()

,StudentID,Name,Gender,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade,Study Hours,Attendance (%),Online Classes Taken
0,1.0,John,Male,85.0,15.0,78.0,1.0,High,80.0,4.8,59.0,False
1,2.0,Sarah,Female,90.0,20.0,85.0,2.0,Medium,87.0,2.2,70.0,True
2,3.0,Alex,Male,78.0,10.0,65.0,0.0,Low,68.0,4.6,92.0,False
3,4.0,Michael,Male,92.0,25.0,90.0,3.0,High,92.0,2.9,96.0,False
4,5.0,Emma,Female,NaN,18.0,82.0,2.0,Medium,85.0,4.1,97.0,True


In [5]:
data.dtypes

StudentID                    float64
Name                          object
Gender                        object
AttendanceRate               float64
StudyHoursPerWeek            float64
PreviousGrade                float64
ExtracurricularActivities    float64
ParentalSupport               object
FinalGrade                   float64
Study Hours                  float64
Attendance (%)               float64
Online Classes Taken          object
dtype: object

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   StudentID                  960 non-null    float64
 1   Name                       966 non-null    object 
 2   Gender                     952 non-null    object 
 3   AttendanceRate             960 non-null    float64
 4   StudyHoursPerWeek          950 non-null    float64
 5   PreviousGrade              967 non-null    float64
 6   ExtracurricularActivities  957 non-null    float64
 7   ParentalSupport            978 non-null    object 
 8   FinalGrade                 960 non-null    float64
 9   Study Hours                976 non-null    float64
 10  Attendance (%)             959 non-null    float64
 11  Online Classes Taken       975 non-null    object 
dtypes: float64(8), object(4)
memory usage: 93.9+ KB


In [7]:
categorical_columns = data.select_dtypes(include=['object']).columns.tolist()

In [8]:
# Check unique values for each categorical column
for col in categorical_columns:
    print(f"\n{col}:")
    print(f"  Unique values: {data[col].nunique()}")
    print(f"  Values: {data[col].unique()[:10]}")


Name:
  Unique values: 962
  Values: ['John' 'Sarah' 'Alex' 'Michael' 'Emma' 'Olivia' 'Daniel' 'Sophia' 'James'
 'Isabella']

Gender:
  Unique values: 2
  Values: ['Male' 'Female' nan]

ParentalSupport:
  Unique values: 3
  Values: ['High' 'Medium' 'Low' nan]

Online Classes Taken:
  Unique values: 2
  Values: [False True nan]


In [9]:
# ============================================================================
# METHOD 1: PANDAS GET_DUMMIES (EASIEST APPROACH)
# ============================================================================

data1 = data.copy()

# Example 1: Encoding 'Gender' variable - CORRECT WAY
print("\n--- Example 1: Encoding 'Gender' with 1 and 0 ---")
print("\nOriginal Gender column (first 10 rows):")
print(data1['Gender'].head(10))

# METHOD A: Using dtype parameter in get_dummies()
print("\n✓ CORRECT: Using dtype=int parameter:")
gender_dummies_int = pd.get_dummies(data1['Gender'], dtype=int)
print(gender_dummies_int.head(10))
print(f"\nData type: {gender_dummies_int.dtypes[0]}")

# Show comparison with original
print("\n✓ Side-by-side comparison:")
gender_comparison = pd.concat([data1['Gender'], gender_dummies_int], axis=1)
print(gender_comparison.head(10))

# Getting k-1 dummy variables with dtype=int
print("\n✓ k-1 dummy variables (drop_first=True) with dtype=int:")
gender_dummies_k_minus_1 = pd.get_dummies(data1['Gender'], drop_first=True, dtype=int)
print(gender_dummies_k_minus_1.head(10))
print(f"Data type: {gender_dummies_k_minus_1.dtypes[0]}")

# Example 2: Encoding 'ParentalSupport' variable
print("\n--- Example 2: Encoding 'ParentalSupport' with 1 and 0 ---")
print("\nOriginal ParentalSupport column (first 10 rows):")
print(data1['ParentalSupport'].head(10))

print("\nUnique values in ParentalSupport:")
print(data1['ParentalSupport'].unique())

print("\n✓ All dummy variables with dtype=int:")
support_dummies = pd.get_dummies(data1['ParentalSupport'], dtype=int)
print(support_dummies.head(10))

print("\n✓ k-1 dummy variables with dtype=int:")
support_dummies_k_minus_1 = pd.get_dummies(data1['ParentalSupport'], drop_first=True, dtype=int)
print(support_dummies_k_minus_1.head(10))

# Example 3: Encoding 'Online Classes Taken'
print("\n--- Example 3: Encoding 'Online Classes Taken' with 1 and 0 ---")
print("\nOriginal 'Online Classes Taken' column (first 10 rows):")
print(data1['Online Classes Taken'].head(10))

print("\n✓ One-Hot Encoding with k-1 dummy variables:")
online_dummies = pd.get_dummies(data1['Online Classes Taken'], drop_first=True, dtype=int)
print(online_dummies.head(10))



--- Example 1: Encoding 'Gender' with 1 and 0 ---

Original Gender column (first 10 rows):
0      Male
1    Female
2      Male
3      Male
4    Female
5    Female
6      Male
7    Female
8      Male
9    Female
Name: Gender, dtype: object

✓ CORRECT: Using dtype=int parameter:
   Female  Male
0       0     1
1       1     0
2       0     1
3       0     1
4       1     0
5       1     0
6       0     1
7       1     0
8       0     1
9       1     0

Data type: int64

✓ Side-by-side comparison:
   Gender  Female  Male
0    Male       0     1
1  Female       1     0
2    Male       0     1
3    Male       0     1
4  Female       1     0
5  Female       1     0
6    Male       0     1
7  Female       1     0
8    Male       0     1
9  Female       1     0

✓ k-1 dummy variables (drop_first=True) with dtype=int:
   Male
0     1
1     0
2     1
3     1
4     0
5     0
6     1
7     0
8     1
9     0
Data type: int64

--- Example 2: Encoding 'ParentalSupport' with 1 and 0 ---

Original Par

In [10]:
online_dummies.head()

,True
0,0
1,1
2,0
3,0
4,1


In [11]:
# ============================================================================
# ENCODING ALL CATEGORICAL VARIABLES WITH dtype=int
# ============================================================================
print("\n--- Encoding ALL Categorical Variables with dtype=int ---")

# Select columns (excluding identifiers)
columns_to_keep = [col for col in data1.columns if col not in ['StudentID', 'Name']]
data_for_encoding = data1[columns_to_keep].copy()

print("\n✓ Applying get_dummies() with dtype=int:")
data_encoded_correct = pd.get_dummies(data_for_encoding, drop_first=True, dtype=int)

print(f"\nOriginal shape: {data_for_encoding.shape}")
print(f"Encoded shape: {data_encoded_correct.shape}")



--- Encoding ALL Categorical Variables with dtype=int ---

✓ Applying get_dummies() with dtype=int:

Original shape: (1000, 10)
Encoded shape: (1000, 11)


In [12]:
data_encoded_correct.head(10)


,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,FinalGrade,Study Hours,Attendance (%),Gender_Male,ParentalSupport_Low,ParentalSupport_Medium,Online Classes Taken_True
0,85.0,15.0,78.0,1.0,80.0,4.8,59.0,1,0,0,0
1,90.0,20.0,85.0,2.0,87.0,2.2,70.0,0,0,1,1
2,78.0,10.0,65.0,0.0,68.0,4.6,92.0,1,1,0,0
3,92.0,25.0,90.0,3.0,92.0,2.9,96.0,1,0,0,0
4,NaN,18.0,82.0,2.0,85.0,4.1,97.0,0,0,1,1
5,95.0,30.0,88.0,1.0,NaN,2.8,97.0,0,0,0,0
6,70.0,8.0,60.0,0.0,62.0,4.5,96.0,1,1,0,0
7,NaN,17.0,77.0,1.0,78.0,1.0,70.0,0,0,1,1
8,82.0,12.0,70.0,2.0,72.0,3.6,50.0,1,1,0,0
9,91.0,22.0,86.0,3.0,88.0,2.9,59.0,0,0,0,1


In [13]:
# Verify data types are integers
data_encoded_correct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   AttendanceRate             960 non-null    float64
 1   StudyHoursPerWeek          950 non-null    float64
 2   PreviousGrade              967 non-null    float64
 3   ExtracurricularActivities  957 non-null    float64
 4   FinalGrade                 960 non-null    float64
 5   Study Hours                976 non-null    float64
 6   Attendance (%)             959 non-null    float64
 7   Gender_Male                1000 non-null   int64  
 8   ParentalSupport_Low        1000 non-null   int64  
 9   ParentalSupport_Medium     1000 non-null   int64  
 10  Online Classes Taken_True  1000 non-null   int64  
dtypes: float64(7), int64(4)
memory usage: 86.1 KB


In [14]:
data_encoded_correct.head()

,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,FinalGrade,Study Hours,Attendance (%),Gender_Male,ParentalSupport_Low,ParentalSupport_Medium,Online Classes Taken_True
0,85.0,15.0,78.0,1.0,80.0,4.8,59.0,1,0,0,0
1,90.0,20.0,85.0,2.0,87.0,2.2,70.0,0,0,1,1
2,78.0,10.0,65.0,0.0,68.0,4.6,92.0,1,1,0,0
3,92.0,25.0,90.0,3.0,92.0,2.9,96.0,1,0,0,0
4,NaN,18.0,82.0,2.0,85.0,4.1,97.0,0,0,1,1


In [15]:
# ============================================================================
# ALTERNATIVE METHOD: Converting Boolean to Integer
# ============================================================================

# Create boolean dummies first (the default behavior)
gender_bool = pd.get_dummies(data1['Gender'], drop_first=True)
print("\nBoolean dummy variables:")
print(gender_bool.head(10))
print(f"Data type: {gender_bool.dtypes[0]}")

# Convert to integer
gender_int = gender_bool.astype(int)
print("\n✓ Converted to integer:")
print(gender_int.head(10))
print(f"Data type: {gender_int.dtypes[0]}")



Boolean dummy variables:
    Male
0   True
1  False
2   True
3   True
4  False
5  False
6   True
7  False
8   True
9  False
Data type: bool

✓ Converted to integer:
   Male
0     1
1     0
2     1
3     1
4     0
5     0
6     1
7     0
8     1
9     0
Data type: int64


In [16]:
# Determine which parameter to use
import sklearn
version_parts = sklearn.__version__.split('.')
major = int(version_parts[0])
minor = int(version_parts[1])

if major > 1 or (major == 1 and minor >= 2):
    USE_NEW_SYNTAX = True
    print(f"✓ Using NEW syntax: sparse_output=False")
else:
    USE_NEW_SYNTAX = False
    print(f"✓ Using OLD syntax: sparse=False")

✓ Using NEW syntax: sparse_output=False


In [17]:
# ============================================================================
# METHOD 2: SCIKIT-LEARN (BACKWARD COMPATIBLE)
# ============================================================================


# Prepare data
string_categorical = ['Gender', 'ParentalSupport', 'Online Classes Taken']
data_sklearn = data[columns_to_keep].copy()

print("\n--- Step 1: Label Encoding (Required for scikit-learn) ---")
label_encoders = {}
for col in string_categorical:
    if col in data_sklearn.columns:
        data_sklearn[col] = data_sklearn[col].fillna('Missing')
        le = LabelEncoder()
        data_sklearn[col] = le.fit_transform(data_sklearn[col].astype(str))
        label_encoders[col] = le
        print(f"\n{col}: {dict(list(zip(le.classes_, le.transform(le.classes_)))[:5])}")

print("\n--- Step 2: One-Hot Encoding ---")

# Create OneHotEncoder with correct parameter based on version
if USE_NEW_SYNTAX:
    try:
        ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
        print("✓ Using sparse_output=False (new syntax)")
    except:
        # Fallback just in case
        ohe = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')
        print("✓ Using sparse=False (old syntax - fallback)")
else:
    ohe = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')
    print("✓ Using sparse=False (old syntax)")

# Fit and transform
encoded_array = ohe.fit_transform(data_sklearn[string_categorical])
feature_names = ohe.get_feature_names_out(string_categorical)

print(f"\nEncoded array shape: {encoded_array.shape}")
print(f"Feature names: {list(feature_names)}")

# Create dataframe
encoded_df = pd.DataFrame(encoded_array, columns=feature_names, index=data_sklearn.index)

# Combine with numerical features
numerical_cols = [col for col in data_sklearn.columns if col not in string_categorical]
data_final_sklearn = pd.concat([data_sklearn[numerical_cols], encoded_df], axis=1)



--- Step 1: Label Encoding (Required for scikit-learn) ---

Gender: {'Female': 0, 'Male': 1, 'Missing': 2}

ParentalSupport: {'High': 0, 'Low': 1, 'Medium': 2, 'Missing': 3}

Online Classes Taken: {'False': 0, 'Missing': 1, 'True': 2}

--- Step 2: One-Hot Encoding ---
✓ Using sparse_output=False (new syntax)

Encoded array shape: (1000, 7)
Feature names: ['Gender_1', 'Gender_2', 'ParentalSupport_1', 'ParentalSupport_2', 'ParentalSupport_3', 'Online Classes Taken_1', 'Online Classes Taken_2']


In [18]:
data_final_sklearn.head(10)

,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,FinalGrade,Study Hours,Attendance (%),Gender_1,Gender_2,ParentalSupport_1,ParentalSupport_2,ParentalSupport_3,Online Classes Taken_1,Online Classes Taken_2
0,85.0,15.0,78.0,1.0,80.0,4.8,59.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,90.0,20.0,85.0,2.0,87.0,2.2,70.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,78.0,10.0,65.0,0.0,68.0,4.6,92.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,92.0,25.0,90.0,3.0,92.0,2.9,96.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,18.0,82.0,2.0,85.0,4.1,97.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5,95.0,30.0,88.0,1.0,NaN,2.8,97.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,70.0,8.0,60.0,0.0,62.0,4.5,96.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
7,NaN,17.0,77.0,1.0,78.0,1.0,70.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
8,82.0,12.0,70.0,2.0,72.0,3.6,50.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
9,91.0,22.0,86.0,3.0,88.0,2.9,59.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
# ============================================================================
# METHOD 3: COLUMN TRANSFORMER (MOST FLEXIBLE)
# ============================================================================

data_ct = data[columns_to_keep].copy() # columns without studentID and Name

# Handle missing values and label encode
for col in string_categorical:    # only selecting the categorical columns (string)
    if col in data_ct.columns:
        data_ct[col] = data_ct[col].fillna('Missing')
        le = LabelEncoder()
        data_ct[col] = le.fit_transform(data_ct[col].astype(str))

# Create the encoder with correct parameter
if USE_NEW_SYNTAX:
    try:
        encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    except:
        encoder = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')
else:
    encoder = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')

# Create ColumnTransformer
ct = ColumnTransformer(
    transformers=[
        ('ohe', encoder, string_categorical)
    ],
    remainder='passthrough'
)

# Transform
data_transformed = ct.fit_transform(data_ct)

# Get feature names
ohe_features = ct.named_transformers_['ohe'].get_feature_names_out(string_categorical) # Returns array like: ['Gender_1', 'Gender_2', 'ParentalSupport_1', 'ParentalSupport_2', ...]
other_features = [col for col in data_ct.columns if col not in string_categorical] # Result: ['AttendanceRate', 'StudyHoursPerWeek', 'Age', 'Salary', ...]
all_features = list(ohe_features) + other_features # Result: Complete list of all column names in order

# Create dataframe
data_final_ct = pd.DataFrame(data_transformed, columns=all_features, index=data_ct.index)



In [20]:
data_final_ct.head(10)

,Gender_1,Gender_2,ParentalSupport_1,ParentalSupport_2,ParentalSupport_3,Online Classes Taken_1,Online Classes Taken_2,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,FinalGrade,Study Hours,Attendance (%)
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,15.0,78.0,1.0,80.0,4.8,59.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,90.0,20.0,85.0,2.0,87.0,2.2,70.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,78.0,10.0,65.0,0.0,68.0,4.6,92.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,92.0,25.0,90.0,3.0,92.0,2.9,96.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN,18.0,82.0,2.0,85.0,4.1,97.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,30.0,88.0,1.0,NaN,2.8,97.0
6,1.0,0.0,1.0,0.0,0.0,0.0,0.0,70.0,8.0,60.0,0.0,62.0,4.5,96.0
7,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN,17.0,77.0,1.0,78.0,1.0,70.0
8,1.0,0.0,1.0,0.0,0.0,0.0,0.0,82.0,12.0,70.0,2.0,72.0,3.6,50.0
9,0.0,0.0,0.0,0.0,0.0,0.0,1.0,91.0,22.0,86.0,3.0,88.0,2.9,59.0


Scikit-learn's one hot encoder class only takes numerical categorical values. So, any value of string type should be label encoded first before one hot encoded.

In the titanic example, the gender of the passengers has to be label encoded first before being one-hot encoded using Scikit-learn's one hot encoder class.

### Count and frequency encoding

- In count encoding we replace the categories by the count of the observations that show that category in the dataset. Similarly, we can replace the category by the frequency -or percentage- of observations in the dataset. That is, if 10 of our 100 observations show the colour blue, we would replace blue by 10 if doing count encoding, or by 0.1 if replacing by the frequency. These techniques capture the representation of each label in a dataset, but the encoding may not necessarily be predictive of the outcome.

- This approach is heavily used in Kaggle competitions, wherein we replace each label of the categorical variable by the count, this is the amount of times each label appears in the dataset. Or the frequency, this is the percentage of observations within that category. The two methods are equivalent.

In [21]:
data2 = data.copy()

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
data2.head()

,StudentID,Name,Gender,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade,Study Hours,Attendance (%),Online Classes Taken
0,1.0,John,Male,85.0,15.0,78.0,1.0,High,80.0,4.8,59.0,False
1,2.0,Sarah,Female,90.0,20.0,85.0,2.0,Medium,87.0,2.2,70.0,True
2,3.0,Alex,Male,78.0,10.0,65.0,0.0,Low,68.0,4.6,92.0,False
3,4.0,Michael,Male,92.0,25.0,90.0,3.0,High,92.0,2.9,96.0,False
4,5.0,Emma,Female,NaN,18.0,82.0,2.0,Medium,85.0,4.1,97.0,True


In [24]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   StudentID                  960 non-null    float64
 1   Name                       966 non-null    object 
 2   Gender                     952 non-null    object 
 3   AttendanceRate             960 non-null    float64
 4   StudyHoursPerWeek          950 non-null    float64
 5   PreviousGrade              967 non-null    float64
 6   ExtracurricularActivities  957 non-null    float64
 7   ParentalSupport            978 non-null    object 
 8   FinalGrade                 960 non-null    float64
 9   Study Hours                976 non-null    float64
 10  Attendance (%)             959 non-null    float64
 11  Online Classes Taken       975 non-null    object 
dtypes: float64(8), object(4)
memory usage: 93.9+ KB


In [25]:
categorical_cols = data2.select_dtypes(include=['object', 'bool']).columns.tolist()


In [26]:
categorical_cols = [col for col in categorical_cols if col not in ['Name', 'StudentID']]


In [27]:
print(f"\nCategorical variables found: {categorical_cols}")



Categorical variables found: ['Gender', 'ParentalSupport', 'Online Classes Taken']


In [30]:
print("\nUnique values in each categorical variable:")
for col in categorical_cols:
    unique_vals = data2[col].unique()
    print(f"\n{col}:")
    print(f"  - Number of unique values: {len(unique_vals)}")
    print(f"  - Values: {unique_vals}")
    print(f"  - Value counts:")
    print(data2[col].value_counts())



Unique values in each categorical variable:

Gender:
  - Number of unique values: 3
  - Values: ['Male' 'Female' nan]
  - Value counts:
Gender
Male      501
Female    451
Name: count, dtype: int64

ParentalSupport:
  - Number of unique values: 4
  - Values: ['High' 'Medium' 'Low' nan]
  - Value counts:
ParentalSupport
High      345
Medium    328
Low       305
Name: count, dtype: int64

Online Classes Taken:
  - Number of unique values: 3
  - Values: [False True nan]
  - Value counts:
Online Classes Taken
True     495
False    480
Name: count, dtype: int64


In [32]:
# let's select relevant features for encoding
# We'll focus on the categorical variables
features_to_encode = categorical_cols.copy()

In [34]:
# Drop rows with missing values in the target variable (FinalGrade)
df_clean = data2.dropna(subset=['FinalGrade'])
print(f"\nDataset shape after removing rows with missing FinalGrade: {df_clean.shape}")



Dataset shape after removing rows with missing FinalGrade: (960, 12)


In [35]:
# Create feature set and target
X = df_clean[features_to_encode].copy()
y = df_clean['FinalGrade']


In [36]:
# Split into train and test sets (70-30 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [37]:
print(f"\nTraining set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")


Training set shape: (672, 3)
Test set shape: (288, 3)


In [39]:
# Original data before encoding
print(X_train.head(10))

     Gender ParentalSupport Online Classes Taken
243  Female          Medium                 True
336  Female            High                False
747  Female            High                False
227  Female             Low                 True
83   Female          Medium                 True
158    Male            High                False
312    Male            High                 True
823  Female          Medium                 True
326    Male            High                False
460    Male            High                False


In [40]:
# Create copies for count encoding
X_train_count = X_train.copy()
X_test_count = X_test.copy()

In [41]:
# Dictionary to store count mappings for each variable
count_maps = {}

In [42]:
for col in features_to_encode:
    print(f"\n--- Processing variable: {col} ---")
    
    # Calculate counts for each category in training set
    count_map = X_train[col].value_counts().to_dict()
    count_maps[col] = count_map
    
    print(f"\nCount mapping for {col}:")
    for category, count in sorted(count_map.items(), key=lambda x: x[1], reverse=True):
        print(f"  '{category}' appears {count} times")
    
    # Apply count encoding to both train and test sets
    X_train_count[col] = X_train[col].map(count_map)
    X_test_count[col] = X_test[col].map(count_map)
    
    # Handle any unseen categories in test set (replace with 0)
    X_test_count[col] = X_test_count[col].fillna(0)
    
    print(f"\nAfter count encoding for {col}:")
    print(f"  Training set - min: {X_train_count[col].min()}, max: {X_train_count[col].max()}")
    print(f"  Test set - min: {X_test_count[col].min()}, max: {X_test_count[col].max()}")



--- Processing variable: Gender ---

Count mapping for Gender:
  'Male' appears 348 times
  'Female' appears 293 times

After count encoding for Gender:
  Training set - min: 293.0, max: 348.0
  Test set - min: 0.0, max: 348.0

--- Processing variable: ParentalSupport ---

Count mapping for ParentalSupport:
  'High' appears 232 times
  'Medium' appears 227 times
  'Low' appears 196 times

After count encoding for ParentalSupport:
  Training set - min: 196.0, max: 232.0
  Test set - min: 0.0, max: 232.0

--- Processing variable: Online Classes Taken ---

Count mapping for Online Classes Taken:
  'True' appears 330 times
  'False' appears 325 times

After count encoding for Online Classes Taken:
  Training set - min: 325.0, max: 330.0
  Test set - min: 0.0, max: 330.0


In [43]:
print("\nFirst 10 rows after COUNT ENCODING:")
print(X_train_count.head(10))


First 10 rows after COUNT ENCODING:
     Gender  ParentalSupport  Online Classes Taken
243   293.0            227.0                 330.0
336   293.0            232.0                 325.0
747   293.0            232.0                 325.0
227   293.0            196.0                 330.0
83    293.0            227.0                 330.0
158   348.0            232.0                 325.0
312   348.0            232.0                 330.0
823   293.0            227.0                 330.0
326   348.0            232.0                 325.0
460   348.0            232.0                 325.0


In [44]:
# Create copies for frequency encoding
X_train_freq = X_train.copy()
X_test_freq = X_test.copy()

In [45]:
# Dictionary to store frequency mappings for each variable
frequency_maps = {}

In [46]:
total_observations = len(X_train)


In [47]:
for col in features_to_encode:
    print(f"\n--- Processing variable: {col} ---")
    
    # Calculate frequencies for each category in training set
    value_counts = X_train[col].value_counts()
    frequency_map = (value_counts / total_observations).to_dict()
    frequency_maps[col] = frequency_map
    
    print(f"\nFrequency mapping for {col}:")
    for category, freq in sorted(frequency_map.items(), key=lambda x: x[1], reverse=True):
        print(f"  '{category}' has frequency {freq:.4f} ({freq*100:.2f}%)")
    
    # Apply frequency encoding to both train and test sets
    X_train_freq[col] = X_train[col].map(frequency_map)
    X_test_freq[col] = X_test[col].map(frequency_map)
    
    # Handle any unseen categories in test set (replace with 0)
    X_test_freq[col] = X_test_freq[col].fillna(0)
    
    print(f"\nAfter frequency encoding for {col}:")
    print(f"  Training set - min: {X_train_freq[col].min():.4f}, max: {X_train_freq[col].max():.4f}")
    print(f"  Test set - min: {X_test_freq[col].min():.4f}, max: {X_test_freq[col].max():.4f}")



--- Processing variable: Gender ---

Frequency mapping for Gender:
  'Male' has frequency 0.5179 (51.79%)
  'Female' has frequency 0.4360 (43.60%)

After frequency encoding for Gender:
  Training set - min: 0.4360, max: 0.5179
  Test set - min: 0.0000, max: 0.5179

--- Processing variable: ParentalSupport ---

Frequency mapping for ParentalSupport:
  'High' has frequency 0.3452 (34.52%)
  'Medium' has frequency 0.3378 (33.78%)
  'Low' has frequency 0.2917 (29.17%)

After frequency encoding for ParentalSupport:
  Training set - min: 0.2917, max: 0.3452
  Test set - min: 0.0000, max: 0.3452

--- Processing variable: Online Classes Taken ---

Frequency mapping for Online Classes Taken:
  'True' has frequency 0.4911 (49.11%)
  'False' has frequency 0.4836 (48.36%)

After frequency encoding for Online Classes Taken:
  Training set - min: 0.4836, max: 0.4911
  Test set - min: 0.0000, max: 0.4911


In [48]:
print("\nFirst 10 rows after FREQUENCY ENCODING:")
print(X_train_freq.head(10))


First 10 rows after FREQUENCY ENCODING:
       Gender  ParentalSupport  Online Classes Taken
243  0.436012         0.337798              0.491071
336  0.436012         0.345238              0.483631
747  0.436012         0.345238              0.483631
227  0.436012         0.291667              0.491071
83   0.436012         0.337798              0.491071
158  0.517857         0.345238              0.483631
312  0.517857         0.345238              0.491071
823  0.436012         0.337798              0.491071
326  0.517857         0.345238              0.483631
460  0.517857         0.345238              0.483631


In [49]:
# Create a comparison dataframe
comparison_df = pd.DataFrame({
    'Index': X_train.head(10).index,
})

for col in features_to_encode:
    comparison_df[f'{col}_Original'] = X_train[col].head(10).values
    comparison_df[f'{col}_Count'] = X_train_count[col].head(10).values
    comparison_df[f'{col}_Frequency'] = X_train_freq[col].head(10).values


In [50]:
print("\nSide-by-side comparison (first 10 rows):")
print(comparison_df.to_string())


Side-by-side comparison (first 10 rows):
   Index Gender_Original  Gender_Count  Gender_Frequency ParentalSupport_Original  ParentalSupport_Count  ParentalSupport_Frequency Online Classes Taken_Original  Online Classes Taken_Count  Online Classes Taken_Frequency
0    243          Female         293.0          0.436012                   Medium                  227.0                   0.337798                          True                       330.0                        0.491071
1    336          Female         293.0          0.436012                     High                  232.0                   0.345238                         False                       325.0                        0.483631
2    747          Female         293.0          0.436012                     High                  232.0                   0.345238                         False                       325.0                        0.483631
3    227          Female         293.0          0.436012              

### Ordinal Encoding

This is the method used to convert categorical variables into numerical values- but only when the categories have a meaningful order.

example:

['Low','Medium','High']

Low - 1
Medium - 2 
High - 3

Similarly: student grades, Educational levels, cancer stages etc.
